##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/federated/tutorials/building_your_own_federated_learning_algorithm"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">	在 Google Colab 中运行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>

## 准备工作

开始之前，请运行以下代码来确保您的环境已正确设置。如果未看到问候语，请参阅[安装](../install.md)指南查看说明。 

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff

**注**：本 Colab 已通过验证，可与[最新发布版本](https://github.com/tensorflow/federated#compatibility)的 `tensorflow_federated` pip 软件包一起使用，但 Tensorflow Federated 项目仍处于预发布开发阶段，可能无法在 `main` 上运行。

# 构建您自己的联合学习算法

在[图像分类](federated_learning_for_image_classification.ipynb)和[文本生成](federated_learning_for_text_generation.ipynb)教程中，您学习了如何针对联合学习 (FL) 设置模型和数据流水线，并通过 TFF 的 `tff.learning` API 层执行联合训练。

这只是 FL 研究的冰山一角。本教程将讨论如何在*不*依赖 `tff.learning` API 的情况下实现联合学习算法。在本教程中，您将完成以下目标：

**目标：**

- 了解联合学习算法的一般结构。
- 探索 TFF 的 *Federated Core*。
- 使用 Federated Core 直接实现联合平均。

虽然本教程为独立教程，但是先查看[图像分类](federated_learning_for_image_classification.ipynb)和[文本生成](federated_learning_for_text_generation.ipynb)教程可能会很有用。


## 准备输入数据

首先加载和预处理包含在 TFF 中的 EMNIST 数据集。有关详情，请参阅[图像分类](federated_learning_for_image_classification.ipynb)教程。

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

为了将数据集馈送到我们的模型中，数据将被展平，每个样本将被转换为 `(flattened_image_vector, label)` 形式的元组。

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

接下来，选择少量客户端，并将上述预处理应用于其数据集。

In [ ]:
client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## 准备模型

这一步将使用与[图像分类](federated_learning_for_image_classification.ipynb)教程相同的模型。此模型（通过 `tf.keras` 实现）具有一个隐藏层，随后是一个 softmax 层。

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

为了在 TFF 中使用此模型，请将 Keras 模型包装为 [`tff.learning.Model`](https://tensorflow.google.cn/federated/api_docs/python/tff/learning/Model)。这使我们可以在 TFF 内执行模型的[前向传递](https://tensorflow.google.cn/federated/api_docs/python/tff/learning/Model#forward_pass)，并[提取模型输出](https://tensorflow.google.cn/federated/api_docs/python/tff/learning/Model#report_local_unfinalized_metrics)。有关详情，另请参阅[图像分类](federated_learning_for_image_classification.ipynb)教程。

In [ ]:
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

虽然上面使用了 `tf.keras` 来创建 `tff.learning.Model`，但 TFF 还支持更多通用模型。这些模型具有以下捕捉模型权重的相关特性：

- `trainable_variables`：与可训练层对应的张量的可迭代对象。
- `non_trainable_variables`：与不可训练层对应的张量的可迭代对象。

在本教程中，将仅使用 `trainable_variables`（因为模型中只有这些！）。

# 构建您自己的联合学习算法

虽然 `tff.learning` API 支持创建联合平均的许多变体，但也有些其他联合算法不适合此框架。例如，您可能想要添加正则化、裁剪或更复杂的算法，例如[联合 GAN 训练](https://github.com/tensorflow/federated/tree/main/tensorflow_federated/python/research/gans)。另外，您可能还对[联合分析](https://ai.googleblog.com/2020/05/federated-analytics-collaborative-data.html)感兴趣。

对于这些更高级的算法，您必须使用 TFF 编写自己的自定义算法。在许多情况下，联合算法包含以下 4 个主要组件：

1. 服务器到客户端的广播步骤。
2. 本地客户端更新步骤。
3. 客户端到服务器的上传步骤。
4. 服务器更新步骤。

在 TFF 中，联合算法通常表示为 [`tff.templates.IterativeProcess`](https://tensorflow.google.cn/federated/api_docs/python/tff/templates/IterativeProcess)（我们始终将其称为 `IterativeProcess`）。这是一个包含 `initialize` 和 `next` 函数的类。在此，`initialize` 将用于初始化服务器，`next` 将执行一个通信轮次的联合算法。我们来为 FedAvg 的迭代过程编写一个框架。

首先，有一个初始化函数，它简单地创建一个 `tff.learning.Model` 并返回其可训练的权重。

In [ ]:
def initialize_fn():
  model = model_fn()
  return model.trainable_variables

这个函数看起来不错，但正如您稍后会看到的，您需要做一点小小的修改，使其成为“TFF 计算”。

接下来，让我们编写一个 `next_fn` 的草图。

In [ ]:
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = client_update(federated_dataset, server_weights_at_client)

  # The server averages these updates.
  mean_client_weights = mean(client_weights)

  # The server updates its model.
  server_weights = server_update(mean_client_weights)

  return server_weights

我们将专注于分别实现这四个组件。首先，我们关注可以在纯 TensorFlow 中实现的部分，即客户端和服务器更新步骤。


## TensorFlow 块 

### 客户端更新

`tff.learning.Model` 可用来以与训练 TensorFlow 模型基本相同的方式进行客户端训练。特别是，可以使用 `tf.GradientTape` 计算批量数据的梯度，然后使用 `client_optimizer` 应用这些梯度。这将只涉及可训练的权重。


In [ ]:
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.trainable_variables
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

### 服务器更新

FedAvg 的服务器更新比客户端更新更简单。本教程将实现普通联合平均，其中服务器模型权重会被替换为客户端模型权重的平均值。同样，此过程只使用可训练的权重。

In [ ]:
@tf.function
def server_update(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.trainable_variables
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

该代码片段可简化为简单地返回 `mean_client_weights`。但是，联合平均的更高级实现可以使用 `mean_client_weights` 和更复杂的技术，例如动量或自适应。

**挑战**：实现用于将服务器权重更新为 model_weights 和 mean_client_weights 的中点的 `server_update` 版本。（注：这种“中点”方式类似于最近在 [Lookahead 优化器](https://arxiv.org/abs/1907.08610)上的工作！）。

到目前为止，只涉及了 TensorFlow 代码。这是有意设计的，因为 TFF 允许您使用许多您已经熟悉的 TensorFlow 代码。接下来，您必须指定<em>编排逻辑</em>，即规定服务器向客户端广播哪些内容，以及客户端向服务器上传哪些内容的逻辑。

这将需要 TFF 的 *Federated Core*。

# Federated Core 简介

Federated Core (FC) 是一组用作 `tff.learning` API 基础的低级接口。不过，这些接口不仅限于学习。事实上，它们可用于对分布式数据进行分析和许多其他计算。

概括来讲，Federated Core 是一个开发环境，可让简洁表达的程序逻辑能够将 TensorFlow 代码与分布式通信算子（例如分布式和与广播）相结合。目标是让研究员和从业者明确控制他们系统中的分布式通信，而不需要系统实现细节（例如指定点对点网络消息交换）。

一个关键点在于，TFF 是专为隐私保护而设计。因此，它允许显式控制数据驻留的位置，以防止在集中式服务器位置不必要地积累数据。

## 联合数据

TFF 中的一个关键概念是“联合数据”，它指的是分布式系统中跨一组设备托管的数据项的集合（例如客户端数据集，或服务器模型权重）。跨所有设备的整个值的集合表示为单个*联合值*。

例如，假设有客户端设备，每个设备都有一个表示传感器温度的浮点数。这些浮点数可以通过以下代码将其表示为*联合浮点数*：

In [ ]:
federated_float_on_clients = tff.FederatedType(tf.float32, tff.CLIENTS)

联合类型由其成员组成（例如 `tf.float32`）的类型 `T` 和一组 `G` 设备指定。通常，`G` 是 `tff.CLIENTS` 或 `tff.SERVER`。此联合类型表示为 `{T}@G`，如下所示。

In [ ]:
str(federated_float_on_clients)

'{float32}@CLIENTS'

为什么 TFF 如此关心布局？TFF 的一个关键目标是能够编写可以部署在真实分布式系统上的代码。这意味着推断哪些设备子集执行哪些代码以及不同的数据段驻留在何处至关重要。

TFF 关注三个信息：*数据*、数据*放置*的位置以及数据如何*转换*。前两个封装在联合类型中，而最后一个封装在*联合计算*中。

## 联合计算

TFF 是一种强类型函数式编程环境，其基本单元是*联合计算*。这些单元是接受联合值作为输入并返回联合值作为输出的逻辑片段。

例如，假设您想要计算客户端传感器上温度的平均值。您可以定义以下代码（使用我们的联合浮点数）：

In [ ]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def get_average_temperature(client_temperatures):
  return tff.federated_mean(client_temperatures)

您可能会问，这和 TensorFlow 中的 `tf.function` 装饰器有什么不同？关键的答案是 `tff.federated_computation` 生成的代码既不是 TensorFlow 也不是 Python 代码；它是以独立于内部平台的*胶水语言*编写的分布式系统规范。

虽然这听起来很复杂，但您可以将 TFF 计算视为具有明确定义的类型签名的函数。可以直接查询这些类型签名。

In [ ]:
str(get_average_temperature.type_signature)

'({float32}@CLIENTS -> float32@SERVER)'

此 `tff.federated_computation` 接受联合类型 `<float>@CLIENTS` 的参数，并返回联合类型 `<float>@SERVER` 的值。联合计算可以从服务器到客户端、从客户端到客户端或者从服务器到服务器。另外，联合计算的构成也可以像普通函数一样，只要它们的类型签名匹配即可。

为了支持开发，TFF 允许您调用 `tff.federated_computation` 作为 Python 函数。例如，您可以调用

In [ ]:
get_average_temperature([68.5, 70.3, 69.8])

69.53334

## 非 Eager 计算和 TensorFlow

有两个关键限制需要注意。首先，当 Python 解释器遇到 `tff.federated_computation` 装饰器时，该函数会被跟踪一次并序列化以备将来使用。由于联合学习的去中心化性质，这种未来用法可能会在别处得到应用，例如远程执行环境。因此，TFF 计算从根本上来说是*非 Eager* 计算。这种行为有点类似于 TensorFlow 中的 [`tf.function`](https://tensorflow.google.cn/api_docs/python/tf/function) 装饰器。

其次，联合计算只能由联合算子（例如 `tff.federated_mean`）组成，不能包含 TensorFlow 运算。TensorFlow 代码必须限制在使用 `tff.tf_computation` 装饰的块中。大多数普通 TensorFlow 代码都可以直接进行装饰，例如下面的函数，它会取一个数字并加 `0.5`。

In [ ]:
@tff.tf_computation(tf.float32)
def add_half(x):
  return tf.add(x, 0.5)

这些也有类型签名，但*没有布局*。例如，您可以调用

In [ ]:
str(add_half.type_signature)

'(float32 -> float32)'

这展示了 `tff.federated_computation` 和 `tff.tf_computation` 之间的重要区别。前者有显式布局，而后者没有。

您可以通过指定布局在联合计算中使用 `tff.tf_computation` 块。我们创建一个加 0.5 的函数，但仅适用于客户端的联合浮点数。我们可以通过使用 `tff.federated_map` 来做到这一点，它会应用给定的 `tff.tf_computation`，同时保留布局。

In [ ]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def add_half_on_clients(x):
  return tff.federated_map(add_half, x)

此函数与 `add_half` 几乎相同，不同之处在于它只接受布局位于 `tff.CLIENTS` 中的值，并返回具有相同布局的值。我们可以在它的类型签名中看到这一点：

In [ ]:
str(add_half_on_clients.type_signature)

'({float32}@CLIENTS -> {float32}@CLIENTS)'

总结：

- TFF 对联合值进行运算。
- 每个联合值都有一个*联合类型*，而联合类型包含*类型*（例如 `tf.float32`）和*布局*（例如 `tff.CLIENTS`）。
- 联合值可以使用*联合计算*进行转换，联合计算必须使用 `tff.federated_computation` 和联合类型签名进行装饰。
- TensorFlow 代码必须包含在带有 `tff.tf_computation` 装饰器的块中。
- 随后可以将这些块合并到联合计算中。


# 再次讨论构建您自己的联合学习算法

既然您已经了解了 Federated Core，您可以构建自己的联合学习算法。请记住，您已经在上面为我们的算法定义了 `initialize_fn` 和 `next_fn`。`next_fn` 将利用您使用纯 TensorFlow 代码定义的 `client_update` 和 `server_update`。

不过，为了使我们的算法成为联合计算，您需要 `next_fn` 和 `initialize_fn` 均为 `tff.federated_computation`。

## TensorFlow 联合块 

### 创建初始化计算

初始化函数将非常简单：您将使用 `model_fn` 创建一个模型。不过，请记住，您必须使用 `tff.tf_computation` 分离出我们的 TensorFlow 代码。

In [ ]:
@tff.tf_computation
def server_init():
  model = model_fn()
  return model.trainable_variables

随后，您可以使用 `tff.federated_value` 将其直接传递到联合计算中。

In [ ]:
@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(), tff.SERVER)

### 创建 `next_fn`

客户端和服务器更新代码现在可用于编写实际算法。首先，将 `client_update` 转换为 `tff.tf_computation`，它接受客户端数据集和服务器权重，并输出更新后的客户端权重张量。

您将需要相应的类型来正确装饰我们的函数。幸运的是，服务器权重的类型可以从我们的模型中直接提取。

In [ ]:
whimsy_model = model_fn()
tf_dataset_type = tff.SequenceType(whimsy_model.input_spec)

我们看一下数据集类型签名。请记住，您获取了 28 x 28 张图像（带整数标签）并已将它们展平。

In [ ]:
str(tf_dataset_type)

'<float32[?,784],int32[?,1]>*'

您还可以使用上面的 `server_init` 函数提取模型权重类型。

In [ ]:
model_weights_type = server_init.type_signature.result

通过检查类型签名，您将能够看到我们模型的架构！

In [ ]:
str(model_weights_type)

'<float32[784,10],float32[10]>'

您现在可以为客户端更新创建我们的 `tff.tf_computation`。

In [ ]:
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update(model, tf_dataset, server_weights, client_optimizer)

服务器更新的 `tff.tf_computation` 版本可以使用您已经提取的类型以类似方式定义。

In [ ]:
@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)

最后，您需要创建 `tff.federated_computation` 将所有这些结合在一起。此函数将接受两个*联合值*，一个对应于服务器权重（采用 `tff.SERVER` 布局），另一个对应于客户端数据集（采用 `tff.CLIENTS` 布局）。

请注意，这两种类型均已在上面定义！您只需要使用 `tff.FederatedType` 为它们提供适当的布局。

In [ ]:
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)

还记得 FL 算法的 4 个元素吗？

1. 服务器到客户端的广播步骤。
2. 本地客户端更新步骤。
3. 客户端到服务器的上传步骤。
4. 服务器更新步骤。

现在，您已经构建了上面的元素，每个部分都可以用一行 TFF 代码简洁地表示。这种简洁性就是为什么您必须格外小心地指定诸如联合类型之类的内容！

In [ ]:
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(
      client_update_fn, (federated_dataset, server_weights_at_client))
  
  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights

您现在有一个 `tff.federated_computation` 用于算法初始化和运行算法的一个步骤。为了完成我们的算法，请将这些内容传递给 `tff.templates.IterativeProcess`。

In [ ]:
federated_algorithm = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn,
    next_fn=next_fn
)

我们看看迭代过程的 <code>initialize</code> 和 `next` 函数的<em>类型签名</em>。

In [ ]:
str(federated_algorithm.initialize.type_signature)

'( -> <float32[784,10],float32[10]>@SERVER)'

这反映了 `federated_algorithm.initialize` 是一个返回单层模型（具有 784×10 权重矩阵和 10 个偏置单元）的无参数函数的事实。

In [ ]:
str(federated_algorithm.next.type_signature)

'(<server_weights=<float32[784,10],float32[10]>@SERVER,federated_dataset={<float32[?,784],int32[?,1]>*}@CLIENTS> -> <float32[784,10],float32[10]>@SERVER)'

在这里，可以看到 `federated_algorithm.next` 接受服务器模型和客户端数据，并返回更新的服务器模型。

## 评估算法

我们来运行几个轮次，看看损失如何变化。首先，您将使用第二个教程中讨论的*集中式*方式定义评估函数。

您将首先创建一个集中的评估数据集，然后应用您用于训练数据的相同预处理。

In [ ]:
central_emnist_test = emnist_test.create_tf_dataset_from_all_clients()
central_emnist_test = preprocess(central_emnist_test)

接下来，您将编写一个接受服务器状态的函数，并使用 Keras 对测试数据集进行评估。如果您熟悉 `tf.Keras`，那么这一切看起来会很熟悉，但要注意 `set_weights` 的用法！

In [ ]:
def evaluate(server_state):
  keras_model = create_keras_model()
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]  
  )
  keras_model.set_weights(server_state)
  keras_model.evaluate(central_emnist_test)

现在，我们初始化我们的算法并对测试集进行评估。

In [ ]:
server_state = federated_algorithm.initialize()
evaluate(server_state)

2042/2042 [==============================] - 2s 767us/step - loss: 2.8479 - sparse_categorical_accuracy: 0.1027


我们训练几个轮次，看看有什么变化。

In [ ]:
for round in range(15):
  server_state = federated_algorithm.next(server_state, federated_train_data)

In [ ]:
evaluate(server_state)

2042/2042 [==============================] - 2s 738us/step - loss: 2.5867 - sparse_categorical_accuracy: 0.0980


损失函数略有下降。虽然跳跃很小，但您只针对一小部分客户端进行了 15 轮训练。为了获得更好的结果，您可能需要进行数百甚至数千轮训练。

## 修改我们的算法

此时此刻，我们停下来思考一下您已经完成的工作。您通过将纯 TensorFlow 代码（用于客户端和服务器更新）与来自 TFF Federated Core 的联合计算相结合，直接实现了联合平均。

为了执行更复杂的学习，您可以简单地改变上面的内容。特别是，通过编辑上面的纯 TF 代码，您可以更改客户端执行训练的方式，或者服务器更新其模型的方式。

**挑战：**将[梯度裁剪](https://towardsdatascience.com/what-is-gradient-clipping-b8e815cdfb48)添加到 `client_update` 函数。


如果您想进行更大的更改，还可以让服务器存储和广播更多数据。例如，服务器还可以存储客户端学习率，并使其随时间衰减！请注意，这将需要更改上述 `tff.tf_computation` 调用中使用的类型签名。

**更高难度的挑战**：在客户端上实现采用学习率衰减的联合平均。

此时，您可能已开始意识到在此框架中实现的内容可以有多大的灵活性。如果想要获得灵感（包括上述更高难度挑战的答案），您可以查看 [`tff.learning.algorithms.build_weighted_fed_avg`](https://tensorflow.google.cn/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg) 的源代码，或查看各种使用 TFF 的[研究项目](https://github.com/google-research/federated)。